In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [3]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [4]:
train.shape

(16278, 18)

In [5]:
val.shape

(5426, 18)

In [6]:
test.shape

(5427, 18)

In [7]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 1, 0)
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 



In [8]:
train = clean(train)
val = clean(val)
test = clean(test)


In [9]:
train['Raised Series A'].value_counts()

0    12302
1      179
Name: Raised Series A, dtype: int64

In [10]:
train.shape , val.shape, test.shape

((12481, 19), (4554, 19), (4568, 19))

In [11]:
train['Investor Names'].isnull().sum()

3578

In [12]:
def split_columns(text):
    text = text.copy()
    # new data frame with split value columns 
    new = text["Organization Industries"].str.split(",", n = 7, expand = True) 
    # making separate first name column from new data frame 
    text["Primary Industry"]= new[0] 
    # making separate last name column from new data frame 
    text["Sub_Ind"]= new[1] 
    text["Sub_Ind2"]= new[2]
    text["Sub_Ind3"]= new[3]
    text["Sub_Ind4"]= new[4]
    text["Sub_Ind5"]= new[5]
    text["Sub_Ind6"]= new[6]
    text["Sub_Ind7"]= new[7]
    # Dropping old Name columns 
    text.drop(columns =["Organization Industries"], inplace = True)

    #same thing but for investor column 
    new1 = text["Investor Names"].str.split(",", n = 11, expand = True) 
    # making separate first name column from new data frame 
    text["Investor1"]= new1[0] 
    # making separate last name column from new data frame 
    text["Investor1"]= new1[1] 
    text["Investor2"]= new1[2]
    text["Investor3"]= new1[3]
    text["Investor4"]= new1[4]
    text["Investor5"]= new1[5]
    text["Investor6"]= new1[6]
    text["Investor7"]= new1[7]
    text["Investor8"]= new1[8]
    text["Investor9"]= new1[9]
    text["Investor10"]= new1[10]
    text["Investor11"]= new1[11]
    # Dropping old Name columns 
    text.drop(columns =["Investor Names"], inplace = True)
    text.drop(columns =["Lead Investors"], inplace = True)
    text.drop(columns =["Number of Investors"], inplace = True)
    text = text[text['Money Raised'].notna()]
    text = text[text['Organization Location'].notna()]
    text = text[text['Primary Industry'].notna()]
    text = text[text['Money Raised Currency (in USD)'].notna()]
    



    return text 



In [13]:
train = split_columns(train)
val = split_columns(val)
test = split_columns(test)


In [14]:
train.head()

,Transaction Name,Transaction Name URL,Organization Name,Organization Name URL,Funding Type,Money Raised,Money Raised Currency,Money Raised Currency (in USD),Announced Date,Total Funding Amount,...,Investor2,Investor3,Investor4,Investor5,Investor6,Investor7,Investor8,Investor9,Investor10,Investor11
19602,Seed Round - Finzy,https://www.crunchbase.com/funding_round/finzy...,Finzy,https://www.crunchbase.com/organization/finzy,Seed,1300000,USD,1300000.0,2018-04-03,2300000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24817,Seed Round - Passbase,https://www.crunchbase.com/funding_round/passb...,Passbase,https://www.crunchbase.com/organization/passbase,Seed,3600000,USD,3600000.0,2019-09-25,4200000,...,Seedcamp,Upheaval Investments,None,None,None,None,None,None,None,None
21855,Seed Round - Flex-Appeal,https://www.crunchbase.com/funding_round/flex-...,Flex-Appeal,https://www.crunchbase.com/organization/flex-a...,Seed,600000,EUR,734076.0,2018-01-18,1585000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8045,Seed Round - Pype,https://www.crunchbase.com/funding_round/pype-...,Pype,https://www.crunchbase.com/organization/pype,Seed,3000000,USD,3000000.0,2019-03-04,5950000,...,None,None,None,None,None,None,None,None,None,None
13954,Seed Round - Kinside,https://www.crunchbase.com/funding_round/kinsi...,Kinside,https://www.crunchbase.com/organization/kinside,Seed,1300000,USD,1300000.0,2018-08-20,4300000,...,Haystack,January Ventures,K50 Ventures,Precursor Ventures,Y Combinator,None,None,None,None,None


In [15]:
train.columns

Index(['Transaction Name', 'Transaction Name URL', 'Organization Name',
       'Organization Name URL', 'Funding Type', 'Money Raised',
       'Money Raised Currency', 'Money Raised Currency (in USD)',
       'Announced Date', 'Total Funding Amount',
       'Total Funding Amount Currency',
       'Total Funding Amount Currency (in USD)', 'Organization Description',
       'Organization Location', 'Raised Series A', 'Primary Industry',
       'Sub_Ind', 'Sub_Ind2', 'Sub_Ind3', 'Sub_Ind4', 'Sub_Ind5', 'Sub_Ind6',
       'Sub_Ind7', 'Investor1', 'Investor2', 'Investor3', 'Investor4',
       'Investor5', 'Investor6', 'Investor7', 'Investor8', 'Investor9',
       'Investor10', 'Investor11'],
      dtype='object')

In [16]:
train.isnull().sum()

Transaction Name                             0
Transaction Name URL                         0
Organization Name                            0
Organization Name URL                        0
Funding Type                                 0
Money Raised                                 0
Money Raised Currency                        0
Money Raised Currency (in USD)               0
Announced Date                               0
Total Funding Amount                         0
Total Funding Amount Currency                0
Total Funding Amount Currency (in USD)       0
Organization Description                     0
Organization Location                        0
Raised Series A                              0
Primary Industry                             0
Sub_Ind                                   1077
Sub_Ind2                                  2551
Sub_Ind3                                  4689
Sub_Ind4                                  6033
Sub_Ind5                                  6887
Sub_Ind6     

In [17]:
train['Raised Series A'].value_counts()

0    7828
1     143
Name: Raised Series A, dtype: int64

In [18]:
train['Money Raised Currency (in USD)'].median()

750000.0

In [19]:
pd.set_option('max_colwidth', 150)
text = train[['Organization Description', 'Organization Name']]
text = text.sort_index()
text= pd.DataFrame(text)

text.head()

,Organization Description,Organization Name
3240,Xperiti is an AI-powered enterprise software designed to give investment management firms a strategic advantage in research.,Xperiti
3241,New Age Meats focuses on rapid development of healthy culitvated meat -- grown from animal cells instead of animal slaughter.,New Age Meats
3251,Gangz is the first digital platform for talent recruitment.,Gangz
3254,ProctorEdu solves personality verification problem during online exams by using audio and video recording and proctor.,ProctorEdu
3255,Parlor is an all-in-one feedback and discovery software for digital product teams.,Parlor


In [20]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [21]:

# Let's take a look at the updated text
data_clean = pd.DataFrame(text['Organization Description'].apply(round1))
data_clean

,Organization Description
3240,xperiti is an aipowered enterprise software designed to give investment management firms a strategic advantage in research
3241,new age meats focuses on rapid development of healthy culitvated meat grown from animal cells instead of animal slaughter
3251,gangz is the first digital platform for talent recruitment
3254,proctoredu solves personality verification problem during online exams by using audio and video recording and proctor
3255,parlor is an allinone feedback and discovery software for digital product teams
...,...
27118,virtai tech focuses on providing global data center class ai accelerator virtualization and resource management software and solutions
27119,experience the stock market as a game pick stocks earn coins get cool prizes
27123,flatfile is a developer of a database software designed to import map and normalize customer data
27125,invygo is an appbased car subscription company that gives customers an access to dealership verified cars


In [22]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [23]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean['Organization Description'].apply(round2))
data_clean

,Organization Description
3240,xperiti is an aipowered enterprise software designed to give investment management firms a strategic advantage in research
3241,new age meats focuses on rapid development of healthy culitvated meat grown from animal cells instead of animal slaughter
3251,gangz is the first digital platform for talent recruitment
3254,proctoredu solves personality verification problem during online exams by using audio and video recording and proctor
3255,parlor is an allinone feedback and discovery software for digital product teams
...,...
27118,virtai tech focuses on providing global data center class ai accelerator virtualization and resource management software and solutions
27119,experience the stock market as a game pick stocks earn coins get cool prizes
27123,flatfile is a developer of a database software designed to import map and normalize customer data
27125,invygo is an appbased car subscription company that gives customers an access to dealership verified cars


In [24]:
data_clean['Organization Name'] = train['Organization Name']
data_clean.set_index('Organization Name')

,Organization Description
Organization Name,
Xperiti,xperiti is an aipowered enterprise software designed to give investment management firms a strategic advantage in research
New Age Meats,new age meats focuses on rapid development of healthy culitvated meat grown from animal cells instead of animal slaughter
Gangz,gangz is the first digital platform for talent recruitment
ProctorEdu,proctoredu solves personality verification problem during online exams by using audio and video recording and proctor
Parlor,parlor is an allinone feedback and discovery software for digital product teams
...,...
VirtAI Tech,virtai tech focuses on providing global data center class ai accelerator virtualization and resource management software and solutions
Fantasy Invest,experience the stock market as a game pick stocks earn coins get cool prizes
Flatfile,flatfile is a developer of a database software designed to import map and normalize customer data


In [25]:
# use this link to start tomorrow: https://stackoverflow.com/questions/37807308/how-can-i-reduce-the-file-size-of-my-ipython-notebook
2+2 


4

In [26]:
# Arrange data into X features matrix and y target vector
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

KeyError: "['tokenized_Description'] not found in axis"

In [27]:
X_train.shape , y_train.shape

NameError: name 'X_train' is not defined

In [28]:
val['Raised Series A'].value_counts()

0    2899
1      13
Name: Raised Series A, dtype: int64

In [29]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100,
                  random_state=42,
                  n_jobs=-1          
)
)

###scores = cross_val_score(pipeline, X_train, y_train, cv=10, 
                         ##scoring = 'average_precision')

###scores_val = cross_val_score(pipeline, X_val, y_val, cv=10, 
                         ##scoring = 'average_precision')'''


pipe = pipeline.fit(X_train, y_train);




NameError: name 'X_train' is not defined

In [30]:
xgb_val = pipe.predict(X_val)
print("XGBoost Classifier val Recall Accuracy:", recall_score(y_val, xgb_val))
print("XGBoost Classifier val Accuracy Score:", accuracy_score(y_val, xgb_val))
print("XGBoost Classifier val Precision Accuracy:", precision_score(y_val, xgb_val))

NameError: name 'pipe' is not defined

In [31]:
# when i ran predict(on X_val, it predicted that 10 of the 18 'Yes' values incorrectly. This could explain why the precision and recall accuracy is so low. 
unique_elements, counts_elements = np.unique(xgb_val, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

NameError: name 'xgb_val' is not defined

In [32]:
#make another 
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

KeyError: "['tokenized_Description'] not found in axis"

In [33]:
#install imbalance package
pip install -U imbalanced-learn

SyntaxError: invalid syntax (<ipython-input-33-15af87e117b1>, line 2)

In [34]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

NameError: name 'X_train' is not defined

In [35]:
#def corpora(d):
    #'Dictionary' Representation of all the words in our corpus
    #id2word = corpora.Dictionary(d["tokenized_Description"])
    #Removing extreme values from the dataset
    #id2word.filter_extremes(no_below=5, no_above=0.95)
    #bag of words(bow) representation of our corpus
    #corpus = [id2word.doc2bow(text) for text in d["tokenized_Description"]]
    #return d 

In [36]:
#train = corpora(train)
#val = corpora(val)
#test = corpora(test)





In [37]:
# precision, recall and F1
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
print('Recall', np.mean(recall), recall)
precision = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='precision')
print('Precision', np.mean(precision), precision)
f1 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')
print('F1', np.mean(f1), f1)

NameError: name 'y_train' is not defined